In [1]:
import gurobipy as gp
from gurobipy import GRB
from operator import itemgetter
import itertools
import math
import numpy as np 
import json
from prettytable import PrettyTable

In [2]:
def formar_asignaciones(profesores_por_curso: dict):
    lista_asignaciones = []
    for curso in profesores_por_curso:
        for tupla in profesores_por_curso[curso]['1']:
            lista_asignaciones.append((curso, tupla[0], tupla[1]))
        for tupla in profesores_por_curso[curso]['M']:
            lista_asignaciones.append((curso, tupla[0], tupla[1]))
    return lista_asignaciones

def formar_asignaciones_multiples(clase_conjunta):
    lista_asignaciones_multiples = []
    cantidad_cursos_simultaneos = {}
    for cs,p,k in clase_conjunta:
        for c in cs:
            lista_asignaciones_multiples.append((c,p,k))
            cantidad_cursos_simultaneos[c,p,k] = len(cs)
    return lista_asignaciones_multiples, cantidad_cursos_simultaneos

def formar_asignaciones_con_restriccion_modulos_diarios(ramos_con_modulos_seguidos: dict, asignaciones: list):
    lista_modulos_seguidos = []
    for curso in ramos_con_modulos_seguidos.keys():
        for ramo in ramos_con_modulos_seguidos[curso]:
            lista_modulos_seguidos.append((curso, ramo))
    asignaciones_con_restriccion_de_modulos_diarios = []
    for elem in asignaciones:
        if (elem[0],elem[2]) in lista_modulos_seguidos:
            asignaciones_con_restriccion_de_modulos_diarios.append(elem)
    return asignaciones_con_restriccion_de_modulos_diarios

def cantidad_maxima_de_dias(carga_profesores, ramos_por_curso):
    tope_dias = {}
    for profesor in carga_profesores:
        suma = 0
        for cantidad_cursos in carga_profesores[profesor]:
            for curso,clase in carga_profesores[profesor][cantidad_cursos]:
                suma += ramos_por_curso[curso][clase]['modulos_semanales'] * int(cantidad_cursos)
            suma += int(len(carga_profesores[profesor][cantidad_cursos]) / int(cantidad_cursos))
        if suma <= 3:
            tope_dias[profesor] = 1
        elif suma <= 6:
            tope_dias[profesor] = 2
        elif suma <= 9:
            tope_dias[profesor] = 3
        elif suma <= 12:
            tope_dias[profesor] = 4
    profesores_con_dias_limitados = []
    for elem in tope_dias.keys():
        profesores_con_dias_limitados.append(elem)
    profesores_con_dias_limitados
    
    return tope_dias, profesores_con_dias_limitados

def asignaciones_no_hechas(profesores_por_curso):
    no_asignaciones = []
    for curso in profesores_por_curso:
        for elem in profesores_por_curso[curso]['0']:
            for nombre in profesores_por_curso[curso]['0'][elem]:
                no_asignaciones.append((curso, nombre, elem))
    return no_asignaciones


In [208]:
model = gp.Model("Generación de horarios de colegio")
# model.setParam('OutputFlag', 1)
f = open('testing_data.json',encoding='utf8')

M = 10000
data = json.load(f)
dias = data['dias']
modulos = data['modulos']
ramos_con_modulos_seguidos = data['ramos_con_modulos_seguidos']
ramos_con_modulos_disjuntos = data['ramos_con_modulos_disjuntos']
horarios = data['horarios']
profesores_por_curso = data['profesores_por_curso']
ramos_por_curso = data['ramos_por_curso']
carga_profesores = data['carga_profesores']
profesores = list(carga_profesores.keys())
multiples_cursos = data['multiples_cursos']
multiples_profesores = data['multiples_profesores']
modulos_consecutivos = data['modulos_consecutivos']
limitaciones_profesores = data['limitaciones_profesores']
# multiples_cursos_y_profesores = data['multiples_cursos_y_profesores']

# # # ! DATOS HECHIZOS
# cantidad_profesores_ramo = {
#     "Lenguaje": 10,
#     "Matemáticas": 10,
#     "Historia": 7,
#     "Artes_Visuales": 5,
#     "Música": 5,
#     "Educación_Física": 8,
#     "Orientación": 4,
#     "Tecnología": 4,
#     "Religión": 5,
#     "Ciencias_Naturales": 7,
#     "Inglés": 7,
#     "Consejo_de_Curso": 1,
#     "Filosofía": 1
# }
# cantidad_cursos_profesor = {
#     "Lenguaje": 4,
#     "Matemáticas": 4,
#     "Historia": 4,
#     "Artes_Visuales": 6,
#     "Música": 6,
#     "Educación_Física": 4,
#     "Orientación": 10,
#     "Tecnología": 10,
#     "Religión": 7,
#     "Ciencias_Naturales": 4,
#     "Inglés": 4,
#     "Consejo_de_Curso": 10,
#     "Filosofía": 10
# }

# import simulador
# ramos_por_curso = simulador.generar_horas_libres(ramos_por_curso)
# lista_profesores_para_asignaciones, profesores = simulador.generar_profesores(7, 74, cantidad_profesores_ramo, cantidad_cursos_profesor)
# profesores_por_curso = simulador.generar_asignaciones(lista_profesores_para_asignaciones, ramos_por_curso)

asignaciones_normales = formar_asignaciones(profesores_por_curso)
no_asignaciones = asignaciones_no_hechas(profesores_por_curso)
total_asignaciones = asignaciones_normales + no_asignaciones
asignaciones_con_restriccion_de_modulos_diarios = formar_asignaciones_con_restriccion_modulos_diarios(
    ramos_con_modulos_seguidos, total_asignaciones
)
# asignaciones_multiples, cantidad_de_cursos_simultaneos = formar_asignaciones_multiples(multiples_cursos)
tope_dias, profesores_con_dias_limitados = cantidad_maxima_de_dias(carga_profesores, ramos_por_curso)


x = model.addVars(total_asignaciones,dias,modulos,vtype=GRB.BINARY, name="x")
s = model.addVars(asignaciones_con_restriccion_de_modulos_diarios,dias,vtype=GRB.BINARY, name="s") # * Solo para los cursos donde se prefieren módulos dobles
w = model.addVars(no_asignaciones, vtype=GRB.BINARY, name="w")
y = model.addVars(profesores_con_dias_limitados,dias,vtype=GRB.BINARY,name="y") # * Solo para los profesores que tienen menos de 5 días

##### Se tiene solo una clase a la vez y se tiene una clase en cada módulo del horario (se consideran los casos donde una clase tiene más de un profesor)

In [209]:
model.addConstrs((sum(x[c,p,r,d,j] for p,r in profesores_por_curso[c]['1']) + sum(x[c,z,l,d,j] for l in profesores_por_curso[c]['0'] for z in profesores_por_curso[c]['0'][l]) <= 1 for c in horarios if profesores_por_curso[c]['1'] != [] for d in horarios[c] for j in horarios[c][d]),name="R1");

In [210]:
model.addConstrs((sum(x[c,p,r,d,j] for p,r in (profesores_por_curso[c]['1']) + profesores_por_curso[c]['M']) + sum(x[c,z,k2,d,j] for z in profesores_por_curso[c]['0'][k2]) >= 1 for c in horarios if profesores_por_curso[c]['M'] != [] for d in horarios[c] for j in horarios[c][d] for k2 in profesores_por_curso[c]['0']),name="R1b");

In [211]:
model.addConstrs((sum(x[c,p,r,d,j] for p,r in (profesores_por_curso[c]['1']) + profesores_por_curso[c]['M']) >= 1 for c in horarios if profesores_por_curso[c]['M'] != [] and profesores_por_curso[c]['0'] == {} for d in horarios[c] for j in horarios[c][d]),name="R1b2");

In [212]:
for c,ps,r in multiples_profesores:
    model.addConstrs((x[c,p1,r,d,j] - x[c,p2,r,d,j] == 0
    for p1,p2 in [(ps[a],ps[b]) for a in range(len(ps)) for b in range(len(ps)) if a < b]
    for d in dias for j in modulos),name=f"R1c[{c},{r}]");

In [213]:
model.addConstrs((sum(x[c,z,r2,d,j] for r2 in profesores_por_curso[c]['0'] for z in profesores_por_curso[c]['0'][r2]) <= 1 for c in horarios if profesores_por_curso[c]['0'] != {} for d in horarios[c] for j in horarios[c][d]),name="RNoProf1");

In [214]:
model.addConstrs((M * w[c,z,r2] >= sum(x[c,z,r2,d,j] for d in horarios[c] for j in horarios[c][d]) for c in horarios for r2 in profesores_por_curso[c]['0'] for z in profesores_por_curso[c]['0'][r2]),name="RNoProf2");

In [215]:
model.addConstrs((sum(w[c,z,r2] for z in profesores_por_curso[c]['0'][r2]) <= 1 \
    for c in horarios for r2 in profesores_por_curso[c]['0']),name="RNoProf3");

##### En el horario se tiene exactamente la cantidad de clases definida

In [216]:
model.addConstrs((sum(x[c,p,r,d,j] for d in dias for j in modulos) == ramos_por_curso[c][r]['modulos_semanales'] for c in profesores_por_curso for p,r in (profesores_por_curso[c]['1'] + profesores_por_curso[c]['M'])),name="R2");

In [217]:
model.addConstrs((sum(x[c,z,r2,d,j] for z in profesores_por_curso[c]['0'][r2] for d in dias for j in modulos) == ramos_por_curso[c][r2]['modulos_semanales'] for c in profesores_por_curso for r2 in profesores_por_curso[c]['0']),name="R2b");

##### No se tiene clases en los módulos que no son parte del horario

In [218]:
model.addConstrs((sum(x[c,p,r,d,j] for p,r in (profesores_por_curso[c]['1'] + profesores_por_curso[c]['M'])) + sum(x[c,z,r2,d,j] for r2 in profesores_por_curso[c]['0'] for z in profesores_por_curso[c]['0'][r2]) == 0 for c in horarios for d in horarios[c] for j in list(set(modulos) - set(horarios[c][d]))),name="R3");

##### Asignación profesores

In [219]:
for cs,p2,r2 in multiples_cursos:
     model.addConstrs((sum(x[c,p,r,d,j] for c,r in carga_profesores[p][q] if c != c2) + x[c2,p,r2,d,j] <= int(q) for c2 in list(set([cs[a] for a in range(len(cs)) for b in range(len(cs))])) for p in list(carga_profesores.keys()) for q in carga_profesores[p] for d in dias for j in modulos if p == p2),name="R4");

In [220]:
model.addConstrs((sum(x[c,p,r,d,j] for q in carga_profesores[p] for c,r in carga_profesores[p][q]) - sum(x[c,p2,r,d,j] * (len(cs)) for cs,p2,r in multiples_cursos if p2 == p if c in cs) <= 1 for p in carga_profesores for q in carga_profesores[p] for d in dias for j in modulos),name="R5");

In [221]:
for cs,p,r in multiples_cursos :
    model.addConstrs((x[c1,p,r,d,j] - x[c2,p,r,d,j] == 0 for c1,c2 in [(cs[a],cs[b]) for a in range(len(cs)) for b in range(len(cs)) if a < b] for d in dias for j in modulos),name="R6");

In [222]:
model.addConstrs((sum(x[c,p,r,d,j] for j in modulos) <= ramos_por_curso[c][r]['maximo_modulos_diarios'] for d in dias for c in horarios for p,r in (profesores_por_curso[c]['1'] + profesores_por_curso[c]['M'])),name="R7");

In [223]:
model.addConstrs((sum(x[c,z,r2,d,j] for j in modulos) <= ramos_por_curso[c][r]['maximo_modulos_diarios'] for d in dias for c in horarios for r2 in profesores_por_curso[c]['0'] for z in profesores_por_curso[c]['0'][r2]),name="R7b");

In [224]:
model.addConstrs((x[c,p,r,d,j] + x[c,p,r,d,z] <= 1 for c,p,r in total_asignaciones if r in ramos_con_modulos_seguidos[c] for d in dias for j,z in list(set([(a,b) for a in horarios[c][d] for b in horarios[c][d] if a < b]) - set([tuple(comb) for comb in modulos_consecutivos]))),name="R8");

In [225]:
model.addConstrs((x[c,p,r,d,j] + x[c,p,r,d,z] <= 1 for c,p,r in total_asignaciones if r in ramos_con_modulos_disjuntos[c] for d in dias for j,z in modulos_consecutivos),name="R9");

In [226]:
model.addConstrs((M * s[c,p,r,d] >= sum(x[c,p,r,d,j] for j in modulos) for c,p,r in asignaciones_con_restriccion_de_modulos_diarios for d in dias),name="RS1");

In [227]:
model.addConstrs((sum(s[c,p,r,d] for d in dias) <= math.ceil(ramos_por_curso[c][r]['modulos_semanales'] / 2) for c in horarios for p,r in (profesores_por_curso[c]['1'] + profesores_por_curso[c]['M']) if r in ramos_con_modulos_seguidos[c]),name="RS2");

In [228]:
model.addConstrs((sum(s[c,z,r2,d] for d in dias) <= math.ceil(ramos_por_curso[c][r2]['modulos_semanales'] / 2) for c in horarios for r2 in profesores_por_curso[c]['0'] for z in profesores_por_curso[c]['0'][r2] if r2 in ramos_con_modulos_seguidos[c]),name="RS3");

In [229]:
model.addConstrs((sum(x[c,p,r,d,j] / int(q) for q in carga_profesores[p] for c,r in carga_profesores[p][q] for d in dias for j in modulos) <= limitaciones_profesores[p]['maximo_modulos'] for p in limitaciones_profesores),name="RCarga1");

In [230]:
model.addConstrs((sum(x[c,p,r,d,j]/int(q) for q in carga_profesores[p] for c,r in carga_profesores[p][q] for j in modulos) <= limitaciones_profesores[p]['maximo_modulos_diario'] for p in limitaciones_profesores for d in dias),name="RCarga2");

In [231]:
model.addConstrs((sum(x[c,p,r,d,j1] + x[c,p,r,d,j2] + x[c,p,r,d,j3] + x[c,p,r,d,j4] + x[c,p,r,d,j5] for q in carga_profesores[p] for c,r in carga_profesores[p][q]) - sum(x[c,p,r,d,j1] * (len(cs) - 1)+ x[c,p,r,d,j2] * (len(cs) - 1) + x[c,p,r,d,j3] * (len(cs) - 1) + x[c,p,r,d,j4] * (len(cs) - 1) + x[c,p,r,d,j5] * (len(cs) - 1) for cs,p2,r in multiples_cursos if p2 == p if c in cs) <= 4 for p in carga_profesores for d in dias for j1,j2,j3,j4,j5 in [(i,i+1,i+2,i+3,i+4) for i in modulos if i+4 in modulos]), name="RModConsecutivos");

##### Tope de días según la cantidad de horas de clases que hace cada profesor

In [232]:
model.addConstrs((M * y[p,d] >= sum(x[c,p,r,d,j] for c,r in carga_profesores[p][q] for j in modulos) for d in dias for p in profesores_con_dias_limitados for q in carga_profesores[p]),name="RY1");

In [233]:
model.addConstrs((sum(y[p,d] for d in dias) <= tope_dias[p] for p in profesores_con_dias_limitados),name="RY2");

In [234]:
obj = sum(x[c,p,r,d,j] + x[c2,p2,r2,d,z] for c,p,r in asignaciones_normales for c2,p2,r2 in asignaciones_normales if p == p2 and c == c2 for d in dias for j,z in modulos_consecutivos)
model.setObjective(obj, GRB.MINIMIZE)

model.write('model.lp')
model.setParam("PoolSolutions", 10)
model.setParam('PoolSearchMode', 2)
model.setParam('OutputFlag', 1)
model.update()

model.computeIIS()
removed =[]
for c in model.getConstrs():
    if c.IISConstr:
        print('%s' % c.constrName)
        # Remove a single constraint from the model
        removed.append(str(c.constrName))
        model.remove(c)
model.optimize()
for i in range(model.SolCount):
    model.setParam("SolutionNumber", i)
    model.update()

    print(model.optimize())
    model.write(f"solutions/out{i+1}.sol")
# model.optimize()
# model.write('out.sol')

Set parameter PoolSearchMode to value 2

Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0     32288         -         0         0         -           0s
        0     21186         -         0         0         -           6s
        0     21186        10         0         0         -          10s
        0     21186        10         0         0         -          15s
        0     20127        10         0         0         -          20s
        0     20127        10         0         0         -          26s
        0     20127        10         0         0         -          31s
        0     19598        10         0         0         -          35s
        0     19069        60         0         0         -          40s
        0     19069        80      

In [354]:
def leer_output(numero_sol):
    lista_final = []
    with open(f'solutions/out{numero_sol}.sol', 'r') as file:
        lines = file.readlines()[2:]
        for line in lines:
            if line[-2] == '1':
                if line[-4] == ']':
                    line = line.strip('\n')
                    lista_final.append(line)
                else:
                    line = line.strip('\n')
                    if round(float(line[line.find(']') + 2:])) == 1:
                        lista_final.append(line[:line.find(']') + 1] + ' 1')
            else:
                if line[-4] != ']':
                    line = line.strip('\n')
                    if int(float(line[line.find(']') + 2:])) == 1:
                        lista_final.append(line[:line.find(']') + 1] + ' 1')

    diccionario_dias = {
        'Lunes': 1,
        'Martes': 2,
        'Miércoles': 3,
        'Jueves': 4,
        'Viernes': 5
    }

    vuelta = {
        1: 'Lunes',
        2: 'Martes',
        3: 'Miércoles',
        4: 'Jueves',
        5: 'Viernes'
    }

    profesores_final = []
    for elem in lista_final:
        if elem[0] == "x":
            elem = elem[2:-3]
            elem = elem.split(',')
            profesores_final.append(elem)
            
    for i in range(len(profesores_final)):
        profesores_final[i][3] = diccionario_dias[profesores_final[i][3]]
    profesores_final = sorted(profesores_final, key= lambda x: x[4])
    profesores_final = sorted(profesores_final, key= lambda x: x[3])
    profesores_final = sorted(profesores_final, key= lambda x: x[0])

    for i in range(len(profesores_final)):
        profesores_final[i][3] = vuelta[profesores_final[i][3]]

    ultimo_curso = None
    ultimo_ramo = None
    ultimo_dia = None
    ultimo_modulo = None
    indices = []
    for i in range(len(profesores_final)):
        if profesores_final[i][0] == ultimo_curso:
            if profesores_final[i][2] == ultimo_ramo:
                if profesores_final[i][3] == ultimo_dia:
                    if profesores_final[i][4] == ultimo_modulo:
                        indices.append(i)
        ultimo_curso = profesores_final[i][0]
        ultimo_ramo = profesores_final[i][2]
        ultimo_dia = profesores_final[i][3]
        ultimo_modulo = profesores_final[i][4]
    indices.reverse()
    for indice in indices:
        profesores_final[indice - 1][1] = profesores_final[indice - 1][1] + ' & ' + profesores_final[indice][1]
        profesores_final.pop(indice)
    return profesores_final

def generar_horario(curso, numero_sol):

    clases_curso = []
    dias_semana = []
    ultimo_dia = None
    horario = PrettyTable(dias_semana)
    lista_auxiliar = []
    lista_dia = None
    for elem in leer_output(numero_sol):    
        if elem[0] == curso:
            clases_curso.append([elem[2], elem[3], elem[4], elem[1]])
            if elem[3] not in dias_semana:
                dias_semana.append(elem[3])
            if elem[3] != ultimo_dia:
                lista_auxiliar.append(lista_dia)
                lista_dia = []
                ultimo_dia = elem[3]
                lista_dia.append(ultimo_dia)
                elem[2] = elem[2].replace('_', ' ')
                elem[1] = elem[1].replace('_', ' ')
                lista_dia.append(f'{elem[2]} \n {elem[1]}')
            else:
                elem[2] = elem[2].replace('_', ' ')
                elem[1] = elem[1].replace('_', ' ')
                lista_dia.append(f'{elem[2]} \n {elem[1]}')

    lista_auxiliar.append(lista_dia)
    lista_auxiliar = lista_auxiliar[1:]
    len_maxima = 0
    for dia in lista_auxiliar:
        if len(dia) > len_maxima:
            len_maxima = len(dia)

    horario.add_column('', [i+1 for i in range(len_maxima - 1)])
    
    for i in range(len(lista_auxiliar)):
        for j in range(len_maxima - len(lista_auxiliar[i])):
            lista_auxiliar[i].append('')
        horario.add_column(lista_auxiliar[i][0], lista_auxiliar[i][1:], align='c')
    horario.format = True   
    print(f'Horario del curso {curso}: ')   
    return horario

curso = '2A'
generar_horario(curso,numero_sol=1)

Horario del curso 2A: 


,Lunes,Martes,Miércoles,Jueves,Viernes
1,Artes Visuales Pérez,Lenguaje Rodríguez,Orientación Mbappé,Lenguaje Rodríguez,Lenguaje Rodríguez
2,Artes Visuales Pérez,Ciencias Naturales Rodríguez,Ciencias Naturales Rodríguez,Ciencias Naturales Rodríguez,Ciencias Naturales Rodríguez
3,Lenguaje Rodríguez,Religión González,Lenguaje Rodríguez,Música González & Hakimi,Historia Rodríguez
4,Matemáticas Pérez,Historia Rodríguez,Matemáticas Pérez,Orientación Mbappé,Educación Física González & Mbappé
5,Matemáticas Pérez,Artes Visuales Pérez,Matemáticas Pérez,Religión González,Educación Física González & Mbappé
6,Lenguaje Rodríguez,Matemáticas Pérez,Educación Física González & Mbappé,Tecnología González,Matemáticas Pérez
7,Historia Rodríguez,Música González & Hakimi,Educación Física González & Mbappé,Lenguaje Rodríguez,Matemáticas Pérez
8,,Lenguaje Rodríguez,Tecnología González,Música González & Hakimi,


In [356]:
def generar_horario_profesor(nombre_profesor,numero_sol):
    clases_profesor = []
    dict_dias = {'Lunes': 1, 'Martes': 2, 'Miércoles': 3, 'Jueves': 4, 'Viernes': 5, 'Sábado': 6, 'Domingo': 7}
    dict_dias_inverso = {1: 'Lunes', 2: 'Martes', 3: 'Miércoles', 4: 'Jueves', 5: 'Viernes', 6: 'Sábado', 7: 'Domingo'}
    for elem in leer_output(numero_sol):
        if elem[1] == nombre_profesor:
            clases_profesor.append(elem)
        elif '&' in elem[1]:
            if nombre_profesor in elem[1]:
                clases_profesor.append([elem[0], nombre_profesor, elem[2], elem[3], elem[4]])
            
    for i in range(len(clases_profesor)):
        clases_profesor[i][3] = dict_dias[clases_profesor[i][3]]
    clases_profesor = sorted(clases_profesor, key= lambda x: x[4])
    clases_profesor = sorted(clases_profesor, key= lambda x: x[3])
    for i in range(len(clases_profesor)):
        clases_profesor[i][3] = dict_dias_inverso[clases_profesor[i][3]]

    ingreso_clase = False
    lista_auxiliar = []
    for i in dias:
        lista_dia = [i]
        for j in modulos:
            if clases_profesor == []:
                lista_dia.append(' ')
            elif clases_profesor != []:
                ingreso_clase = False
                primera = True
                if i != clases_profesor[0][3] or j != int(clases_profesor[0][4]):
                    lista_dia.append(' ')          
                while i == clases_profesor[0][3] and j == int(clases_profesor[0][4]):
                    ingreso_clase = True
                    if primera:
                        cursos = f'{clases_profesor[0][0]}'
                    else:
                        cursos += f' & {clases_profesor[0][0]}'
                    ramo = clases_profesor[0][2]
                    clases_profesor.pop(0)
                    primera = False
                    if clases_profesor == []:
                        break
                if ingreso_clase:
                    lista_dia.append(f'{cursos} \n {ramo}')
        lista_auxiliar.append(lista_dia)

    # print(dias)
    horario = PrettyTable([])
    horario.add_column('', modulos)
    # print(modulos)
    # print(horario)
    for i in range(len(dias)):
        # print(lista_auxiliar)
        horario.add_column(lista_auxiliar[i][0], lista_auxiliar[i][1:], align='c')
    horario.format = True   
    print(f'Horario del profesor {nombre_profesor}: ')   
    return horario

generar_horario_profesor(nombre_profesor='Mbappé', numero_sol=1)

Horario del profesor Mbappé: 


,Lunes,Martes,Miércoles,Jueves,Viernes
1,1B Artes_Visuales,,2A & 2B Orientación,1B Tecnología,
2,,,,,
3,,,,1B Orientación,
4,,1B Artes_Visuales,2B Tecnología,2A & 2B Orientación,2A & 2B Educación_Física
5,,1B Artes_Visuales,1B Orientación,,2A & 2B Educación_Física
6,,1B Educación_Física,2A & 2B Educación_Física,1B Tecnología,1B Educación_Física
7,,1B Educación_Física,2A & 2B Educación_Física,,1B Educación_Física
8,,,,2B Tecnología,
9,,,,,


In [73]:
def generar_output():
    output = {}
    ultimo_elem = None
    ultimo_dia = None
    ultimo_modulo = None
    for elem in leer_output(numero_sol=1):
        if elem[0] == ultimo_elem:
            if elem[3] == ultimo_dia:
                if elem[4] == ultimo_modulo:
                    print(elem)
                    output[elem[0]][elem[3]][elem[4]] += ',' + elem[1]
                elif elem[4] != ultimo_modulo:
                    output[elem[0]][elem[3]][elem[4]] = elem[2] + ',' + elem[1]
            elif elem[3] != ultimo_dia:
                output[elem[0]][elem[3]] = {}
                output[elem[0]][elem[3]][elem[4]] = elem[2] + ',' + elem[1]
        else:
            output[elem[0]] = {}
            output[elem[0]][elem[3]] = {}
            output[elem[0]][elem[3]][elem[4]] = elem[2] + ',' + elem[1]
        ultimo_elem = elem[0]
        ultimo_dia = elem[3]
        ultimo_modulo = elem[4]
    return output

### A continuación se presenta una idea donde se podría seleccionar ciertas componentes del horario y fijarlas o bloquearlas para una siguiente iteración

##### Una idea sería que, si se selecciona un bloque a la vez, uno puede fijarlo en esa posición o bloquearlo de esa posición

In [ ]:
asignaciones_vetadas = []
# ! Esta sería información que se obtiene clickeando el bloque
def vetar_asignacion(curso, profesor, clase, dia, modulo, asignaciones_vetadas):
    asignaciones_vetadas.append((curso, profesor, clase, dia, modulo))

def fijar_asignacion(curso, profesor, clase, dia, modulo, asignaciones_fijadas):
    asignaciones_fijadas.append((curso, profesor, clase, dia, modulo))

curso_vetado = '1A'
profesor_vetado = 'Pérez'
clase_vetada = 'Matemáticas'
dia_vetado = 'Lunes'
modulo_vetado = 2
asignaciones_vetadas.append((curso_vetado, profesor_vetado, clase_vetada, dia_vetado, modulo_vetado))
# * y después esto se agrega al modelo
model.addConstrs((x[c,p,k,d,j] == 0 for c,p,k,d,j in asignaciones_vetadas),name="RExtra1");

asignaciones_fijadas = []
curso_fijado = '1A'
profesor_fijado = 'González'
clase_fijada = 'Educación_Física'
dia_fijado = 'Martes'
modulo_fijado = 5
model.addConstrs((x[c,p,k,d,j] == 1 for c,p,k,d,j in asignaciones_fijadas),name="RExtra2");

##### También se puede querer mantener/vetar un comportamiento de dos partes del horario simultáneamente.

In [ ]:
# * Si se tienen dos cursos seleccionados se puede querer que en un siguiente horario estén juntos sí o sí. Esto obviamente no se aplicaría para el caso que sean clases que
# * necesariamente son consecutivas

combinaciones_vetadas = []

curso_a_tratar = '1A'
profesor1 = 'Rodríguez'
profesor2 = 'Pérez'
clase1 = 'Lenguaje'
clase2 = 'Matemáticas'
# ? Interesa realmente módulo y día

combinaciones_vetadas.append((curso_a_tratar, profesor1, profesor2, clase1, clase2))

# ! Para vetarlos
model.addConstrs((x[c,p1,k1,d,j] + x[c,p2,k2,d,z] <= 1 for c,p1,p2,k1,k2 in combinaciones_vetadas for d in dias for j,z in modulos_consecutivos),name="RExtra3");

def vetar_combinaciones(curso, profesor1, profesor2, clase1, clase2):
    combinaciones_vetadas.append((curso, profesor1, profesor2, clase1, clase2))

combinaciones_deseadas = []

curso_a_tratar = '1A'
profesor1 = 'González'
profesor2 = 'Pérez'
clase1 = 'Música'
clase2 = 'Matemáticas'
dia = 'Miércoles'
modulo1 = 1
modulo2 = 2

combinaciones_deseadas.append((curso_a_tratar,profesor1,profesor2,clase1,clase2,dia,modulo1,modulo2))

# * Para fijarlos
model.addConstrs((x[c,p1,k1,d,j1] + x[c,p2,k2,d,j2] == 2 for c,p1,p2,k1,k2,d,j1,j2 in combinaciones_deseadas),name="RExtra5");

In [ ]:
def actualizar_restricciones(modelo: gp.Model, asignaciones_fijadas: list, asignaciones_vetadas: list, combinaciones_vetadas: list):
    modelo.addConstrs((x[c,p,k,d,j] == 1 for c,p,k,d,j in asignaciones_fijadas), name="Fijadas")
    modelo.addConstrs((x[c,p,k,d,j] == 0 for c,p,k,d,j in asignaciones_vetadas), name="Vetadas")
    modelo.addConstrs((x[c,p1,k1,d,j] + x[c,p2,k2,d,z] <= 1 for c,p1,p2,k1,k2 in combinaciones_vetadas for d in dias for j,z in modulos_consecutivos),name="CombVetadas")
    return modelo

En el caso que no se tenga un profesor fijado y se quiera que el programa elija se tiene que tener una lista de profesores que puedan dictar esa clase (para ese curso)

In [244]:

# Esta variable t ayuda a que si no hay un profesor asignado se asigne uno dependiendo qué conviene más (en base a disponibilidad y otras restricciones)